In [28]:
from ipywidgets import Layout, Button, Box, FloatText,Text, Password, Dropdown, Label, IntSlider,DatePicker
from IPython.display import clear_output
import datetime

if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/trupa/pnl-analysis
    %cd pnl-analysis/

api_key_in = Password()
api_secret_in = Password()
api_passphrase_in = Password()
api_group_in = Text()
start_date_in = DatePicker(disabled=False,value=datetime.datetime.now())
end_date_in = DatePicker(disabled=False,value=datetime.datetime.now())

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_passphrase_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

op_item_group_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

box_passPhrase = Box([Label(value='Passphrase'), 
    api_passphrase_in], layout=op_item_passphrase_layout)
box_group =   Box([Label(value='Api group'), 
    api_group_in], layout=op_item_group_layout)

form_items = [
    Box([Label(value='Api Key'), 
         api_key_in], layout=form_item_layout),
    Box([Label(value='Secret Key'), 
         api_secret_in], layout=form_item_layout),
    box_passPhrase,
    box_group,
    Box([Label(value='Start Date'), start_date_in], layout=form_item_layout),
    Box([Label(value='End Date'), end_date_in], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

box_passPhrase.layout.display = 'none'
box_group.layout.display = 'flex'
display(form)

def get_exchange_data():

    import pandas as pd
    import numpy as np
    import json
    import plotly.graph_objects as go
    from IPython.core.display import display, HTML
    from datetime import datetime

    # Set display
    pd.options.display.float_format = '{:,.10f}'.format

    START_TIME=start_date_in.value
    END_TIME=end_date_in.value
    api_key = api_key_in.value
    api_secret = api_secret_in.value
    api_group = api_group_in.value
    client = None
    start_dt = int(pd.to_datetime(START_TIME).timestamp()*1000)
    end_dt = int(pd.to_datetime(END_TIME).timestamp()*1000)

    from src.ascendex.AscendexClientWrapper import AscendexClientWrapper
    client = AscendexClientWrapper.createInstance(api_key,api_secret,api_group)

    from jinja2 import Template
    
    class Printer:
        @staticmethod
        def h1(*title):
            title_t = Template("<h1>{{title}}</h1>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h2(*title):
            title_t = Template("<h2>{{title}}</h2>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))
        @staticmethod
        def h3(*title):
            title_t = Template("<h3>{{title}}</h3>")
            display(HTML(title_t.render(title=" ".join([str(t) for t in title]))))

        @staticmethod
        def html(html):
            display(HTML(html))
        
        @staticmethod
        def p_df(df):
            """Neatly display a dataframe"""
            display(HTML(df.to_html()))
            
        def p_dict(dt):
            df = pd.DataFrame()
            df['_'] = dt.keys()
            df['__'] = dt.values()
            Printer.p_df(df)

    df_trades = client.get_all_trades(start_dt,end_dt)
    df_balances = client.get_current_asset_balances()

    return df_trades, df_balances


Box(children=(Box(children=(Label(value='Api Key'), Password()), layout=Layout(display='flex', flex_flow='row'…

In [29]:
from src.processing import pnl_calculate
from src.processing import calc_trading_fees

df_trades, df_balances = get_exchange_data()
total_fee, commissions = calc_trading_fees(df_trades)


Exception: (404, 'The requested resource could not be found.')

In [7]:
df_trades

,symbol,price,qty,quoteQty,commission,commissionAsset,side,commissionAssetUsdPrice,date_time
0,GT/USDT,6.6831000000,23.1000000000,154.3796100000,0.2778832980,USDT,sell,1.0000000000,2022-01-30 03:59:32.544
1,EZ/USDT,2.4570000000,67.0800000000,164.8155600000,0.3296311200,USDT,sell,1.0000000000,2022-01-30 03:55:47.320
2,EZ/USDT,2.4590000000,67.2800000000,165.4415200000,0.2977947360,USDT,buy,1.0000000000,2022-01-30 03:55:35.968
3,CRWNY/USDT,0.0082570000,"2,778.0000000000",22.9379460000,0.0458758920,USDT,sell,1.0000000000,2022-01-30 03:51:07.524
4,CNTR/USDT,0.0008120000,"161,773.0000000000",131.3596760000,0.2364474170,USDT,sell,1.0000000000,2022-01-30 03:44:44.881
...,...,...,...,...,...,...,...,...,...
1098,MITX/USDT,0.0147390000,"4,723.0000000000",69.6122970000,0.1253021350,USDT,sell,1.0000000000,2022-01-28 00:16:08.060
1099,LOOKS/USDT,4.0680000000,4.2900000000,17.4517200000,0.0314130960,USDT,sell,1.0000000000,2022-01-28 00:14:21.158
1100,D11/USDT,0.0354900000,"3,996.0000000000",141.8180400000,0.2552724720,USDT,buy,1.0000000000,2022-01-28 00:11:31.388
1101,D11/USDT,0.0355800000,"4,163.0000000000",148.1195400000,0.2666151720,USDT,sell,1.0000000000,2022-01-28 00:11:10.693


In [31]:
df_trades.symbol.unique()

array(['GT/USDT', 'EZ/USDT', 'CRWNY/USDT', 'CNTR/USDT', 'PRXY/USDT',
       'ADEL/USDT', 'FIN/USDT', 'CPOOL/USDT', 'VERA/USDT', 'AKT/USDT',
       'LOOKS/USDT', 'NEWO/USDT', 'D11/USDT', 'JET/USDT', 'GARI/USDT',
       'PLN/USDT', 'LUSD/USDT', 'MAHA/USDT', 'BOLT/USDT', 'ABBC/USDT',
       'ATLAS/USDT', 'HPB/USDT', 'TIME/USDT', 'PKF/USDT', 'MITX/USDT',
       'REVV/USDT', 'CAPS/USDT', 'STF/USDT', 'FINE/USDT', 'CET/USDT',
       'TARA/USDT', 'DIVI/USDT', 'OIN/USDT', 'RNDR/USDT', 'NIOX/USDT',
       'MONI/USDT', 'ASD/USDT'], dtype=object)

In [32]:
df_buys = df_trades[df_trades["side"] == "buy"]
df_sells = df_trades[df_trades["side"] == "sell"]
num_trades = len(df_trades)
num_buys = len(df_buys)
num_sells = len(df_sells) 
quote_proceeds = df_sells["quoteQty"].sum()
quote_spent = df_buys["quoteQty"].sum()
quote_delta = quote_proceeds - quote_spent


In [25]:
df_buys["quoteQty"].mean()

110.73304442477877

In [24]:
df_sells["quoteQty"].mean()

78.05053864516128

In [3]:
import pandas as pd
from src.ascendex.AscendexClientWrapper import AscendexClientWrapper
client = AscendexClientWrapper.createInstance('mlRhejXWnpwO1kGRkbUccNSvAqrNmRmK','uCsdEGTX3MjUjpFye6PsHFhi7k5Rb9C6RIoikWimPzAd2MBMKPVcALXQntok6mVU','3')
START_TIME=start_date_in.value
END_TIME=end_date_in.value
start_dt = int(pd.to_datetime(START_TIME).timestamp()*1000)
end_dt = int(pd.to_datetime(END_TIME).timestamp()*1000)
rs = client.client.getHistOrders(account='cash', startTime=start_dt,endTime=end_dt)
df_res = pd.DataFrame(rs)

